# 예제 3.49: 모델 상태(state_dict) 저장

## 학습목표
1. **state_dict()** 의 개념 이해하기 - 모델의 학습 가능한 파라미터만 저장
2. **torch.save()로 state_dict 저장** 방법 익히기
3. **모델 전체 저장 vs state_dict 저장** 차이점 파악하기
4. **state_dict 저장의 장점** 이해하기 (파일 크기, 호환성)

---

#### 라이브러리 및 클래스 정의

**state_dict란?**
- 모델의 학습 가능한 파라미터(weight, bias)를 담은 딕셔너리
- 모델 구조는 저장하지 않음 → 불러올 때 모델 클래스 필요
- 파일 크기가 작고, 다른 환경에서도 호환성이 좋음

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    """커스텀 데이터셋 클래스"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        # 비선형 데이터를 [x², x] 형태로 변환
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    """커스텀 모델 클래스"""
    
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 데이터 및 모델 준비

In [ ]:
# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델, 손실함수, 옵티마이저 설정
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 학습 루프

In [ ]:
# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)          # 순전파
        loss = criterion(output, y)  # 손실 계산

        optimizer.zero_grad()  # 기울기 초기화
        loss.backward()        # 역전파
        optimizer.step()       # 파라미터 업데이트

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

---

#### 모델 추론 및 상태 저장

In [ ]:
# 추론 수행
with torch.no_grad():  # 기울기 계산 비활성화
    model.eval()  # 평가 모드
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],    # x=1일 때
            [5 ** 2, 5],    # x=5일 때
            [11 ** 2, 11]   # x=11일 때
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

In [ ]:
# state_dict 저장 (권장 방법)
# model.state_dict(): 모델의 학습 가능한 파라미터만 반환
# 장점: 파일 크기 작음, 다른 환경에서 호환성 좋음
# 단점: 불러올 때 모델 클래스 정의 필요
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)
print("모델 상태(state_dict) 저장 완료!")